In [2]:
using WebDisplay

In [3]:
setup_ijulia()

HTML{String}("<script>(function () {\n\n    var contexts = {};\n\n    function makeContext(id, data, sendCallback, dom, commands)\n    {\n        var ctx = {\n            type: \"context\",\n            id: id,\n            data: data,\n            sendCallback: sendCallback,\n            dom: dom,\n            commands: commands || {}\n        }\n\n        contexts[id] = ctx;\n\n        return ctx;\n    }\n\n\n    function createNode(context, data, parentNode)\n    {\n        var nodeType = data.nodeType;\n        return WebDisplay.NodeTypes[nodeType]\n               .create(context, data, parentNode)\n    }\n\n    function getHandler(ctx, cmd)\n    {\n        debugger;\n        var f = ctx.commands[cmd];\n        if (typeof f !== \"undefined\") {\n            return f;\n        }\n        var parts = cmd.split('.');\n        var inherit = WebDisplay.CommandSets;\n        if (inherit && inherit[parts[0]]) {\n            f = inherit[parts[0]][parts[1]]\n            if (typeof f !== \"undefined\") {\n                return f;\n            }\n        }\n        return function (ctx, data) {\n            console.warn(\"No handler found for \" + cmd, \" Conext: \", ctx)\n        }\n    }\n\n\n    function dispatch(msg)\n    {\n        if (msg.type != \"command\") {\n            console.warn(\"invalid message received\", msg)\n        } else {\n            var ctx = contexts[msg.context];\n            var f = getHandler(ctx, msg.command);\n            f(ctx, msg.data)\n        }\n    }\n\n    function mount(targetQuery, data)\n    {\n        // every root element gets a context by default\n        var context = makeContext(targetQuery, data, WebDisplay.sendCallback)\n        var target;\n\n        if (targetQuery) {\n            target = document.querySelector(targetQuery);\n\n            while (target.firstChild) {\n                target.removeChild(target.firstChild);\n            }\n        }\n\n        var node = createNode(context, data, target);\n        context.dom = node;\n\n        if (target) {\n            target.appendChild(node);\n        }\n\n        return node\n    }\n\n    function send(ctx, cmd, data)\n    {\n        ctx.sendCallback(message(ctx, cmd, data));\n    }\n\n    function message(ctx, cmd, data)\n    {\n        return {\n            type: \"message\",\n            context: ctx.id,\n            command: cmd,\n            data: data\n        }\n    }\n\n\n    function sendNotSetUp()\n    {\n        console.error(\"WebDisplay.sendCallback not set up\")\n    }\n\n    window.WebDisplay = {\n        type: \"WebDisplay\",\n\n        // For Base.show or a package to create an element.\n        mount: mount,\n\n        // Create Context - for use by NodeTypes\n        makeContext: makeContext,\n\n        // createNode\n        createNode: createNode,\n\n        // For Providers to call when commands are received from Julia\n        dispatch: dispatch,\n\n        // Send a message back to the Context on Julia\n        send: send,\n\n        // given by Provider, to be called when JS needs to send a command to Julia\n        sendCallback: sendNotSetUp\n    };\n})();\n\n// TODO: have many instances of WebDisplay\n// problem: send_callback needs to be used for construction\n// where do components \"register_handler\"?\n</script>")

HTML{String}("<script>(function () {\n\nvar STYLE_KEY = 'style';\nvar EVENT_KEY = 'events';\nvar ATTR_KEY = 'attributes';\nvar ATTR_NS_KEY = 'attributes_ns';\n\nfunction applyProps(context, domNode, props)\n{\n    for (var key in props) {\n        var value = props[key];\n\n        switch (key)\n        {\n            case STYLE_KEY:\n                applyStyles(domNode, value);\n                break;\n\n            case EVENT_KEY:\n                applyEvents(domNode, context, value);\n                break;\n\n            case ATTR_KEY:\n                applyAttrs(domNode, value);\n                break;\n\n            case ATTR_NS_KEY:\n                applyAttrsNS(domNode, value);\n                break;\n\n            case 'value':\n                // special-case for updating value only\n                // when it has changed.\n                if (domNode[key] !== value)\n                {\n                    domNode[key] = value;\n                }\n                break;\n\n            default:\n                domNode[key] = value;\n                break;\n        }\n    }\n}\n\nfunction applyStyles(domNode, styles)\n{\n    var domNodeStyle = domNode.style;\n\n    for (var key in styles)\n    {\n        domNodeStyle[key] = styles[key];\n    }\n}\n\nfunction applyEvents(domNode, context, events)\n{\n    var allHandlers = domNode.webdisplay_handlers || {};\n\n    for (var key in events)\n    {\n        var handler = allHandlers[key];\n        var value   = events[key];\n\n        if (typeof value === 'undefined')\n        {\n            domNode.removeEventListener(key, handler);\n            allHandlers[key] = undefined;\n        }\n        else if (typeof handler === 'undefined')\n        {\n            var handler = makeEventHandler(context, value);\n            domNode.addEventListener(key, function (event) {\n                handler.call(domNode, event);\n            });\n            allHandlers[key] = handler;\n        }\n        else\n        {\n            handler.options = value;\n        }\n    }\n\n    domNode.webdisplay_handlers = allHandlers;\n}\n\nvar defaultEventOptions = {\n    stopPropagation: true,\n    preventDefault: false\n}\n\nfunction makeEventHandler(context, value)\n{\n    console.log(\"makeEventHandler\", context, value);\n\n    if (typeof value == \"string\") {\n        return makeEventHandler(context, {\n            code: value, options: defaultEventOptions\n        })\n    }\n\n    var options = value.options;\n    var code = value.code;\n\n    function eventHandler(event) {\n        if (options.stopPropagation) {\n            event.stopPropagation();\n        }\n        if (options.preventDefault) {\n            event.preventDefault();\n        }\n    debugger;\n\n        // http://stackoverflow.com/questions/1271516/executing-anonymous-functions-created-using-javascript-eval\n        var f = new Function(\"event\", \"context\", \"(\" + code + \").call(this, event, context)\");\n        f.call(this, event, context);\n    }\n\n    eventHandler.options = options;\n    return eventHandler;\n}\n\nfunction appendChildren(context, parentNode, children) {\n    if (children) {\n        for (var nChildren=children.length, i=0; i<nChildren;i++) {\n            if (typeof children[i] === \"string\") {\n                parentNode.appendChild(document.createTextNode(children[i]));\n            } else {\n                var child = WebDisplay.createNode(context, children[i], parentNode);\n                parentNode.appendChild(child);\n            }\n        }\n    }\n\n}\n\nvar namespaces = {svg: \"http://www.w3.org/2000/svg\"}\n\nfunction createDOM(ctx, data, parentNode) {\n\n    var args = data.instanceArgs;\n    var dom;\n    if (args.namespace == \"html\") {\n        dom = document.createElement(args.tag);\n    } else {\n        var ns = namespaces[args.namespace] || args.namespace;\n        dom = document.createElementNS(ns, args.tag);\n    }\n\n    applyProps(ctx, dom, data.props);\n    appendChil

HTML{String}("<script>(function (IPython, \$, WebDisplay) {\n\n    function initComm(notebook)\n    {\n        var commManager = notebook.kernel.comm_manager;\n\n        // Register a \"target\" so that Julia can create a Comm\n        // to communicate.\n        commManager.register_target(\"webdisplay_comm\",\n            function (comm) {\n                WebDisplay.sendCallback = function (msg) { comm.send(msg); }\n                comm.on_msg(function (msg) {\n                    WebDisplay.dispatch(msg.content.data);\n                });\n            }\n        );\n    }\n\n    \$(document).ready(function() {\n\n        try {\n            // try to initialize right away - note: \$.ready is async.\n            initComm(IPython.notebook);\n        } catch (e) {\n            // wait on the status_started event.\n            \$([IPython.events]).on(\n                'kernel_created.Kernel kernel_created.Session',\n                 function(event, notebook) { initComm(notebook); }\n            );\n        }\n    });\n\n})(IPython, jQuery, WebDisplay);\n</script>")

In [4]:
Node(:div,
      Node(:h1, "Clickbait!", className="title", style=Dict(:color=>"red")),
      Node(:p, "You will not believe what happens when you click this link!", className="description"),
      className="article"
)

WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:div),Persistent{WebDisplay.Node{WebDisplay.DOM}}[WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:h1),Persistent{String}[Clickbait!],Dict{Symbol,Any}(Pair{Symbol,Any}(:style,Dict(:color=>"red")),Pair{Symbol,Any}(:className,"title")),nothing,1), WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:p),Persistent{String}[You will not believe what happens when you click this link!],Dict(:className=>"description"),nothing,1)],Dict(:className=>"article"),nothing,4)

In [5]:
Node(
    :div, "show my messages",
    events=Dict(
      "click" => "function () { alert('Nice, you have no messages.'); }"
    )
)

WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:div),Persistent{String}[show my messages],Dict(:events=>Dict("click"=>"function () { alert('Nice, you have no messages.'); }")),nothing,1)

In [8]:
function counter(count=0)
  makecontext() do ctx
    # ctx is the Context object

    # handle(context, command_name) - add a handler for a command coming from JavaScript

    handle(ctx, :change) do d
        send(ctx, :set_count, count+=d)
    end

    # handlejs(ctx, command_name, function_string) - add a handler for a command coming in from the Julia side
    # in this case we access the element with the id "count" from ctx.dom, and set its contents to the new count

    handlejs(ctx, :set_count, "function (ctx,msg) {ctx.dom.querySelector('#count').textContent = msg}")

    # the btn function defined below takes a label and a change value and creates a button which
    # when clicked, asks julia to change the counter by the given change value by sending the "change" command
    # recall that we have added the handler for :change using `handle` above.

    btn(label, change) = Node(:button, label,
            events=Dict(
                # an event handler on Javascript always gets two arguments: the event and the context
                # event is object passed by JavaScript into the event handler, and context is the context
                # using which you can talk to julia or access and modify contents of the context (context.dom as seen above)
                "click"=>"function (event,ctx) { WebDisplay.send(ctx, 'change', $change) }"
            )
        )

    # This function should return the DOM object that is watched over by the context
    # for relaying messages.

    Node(:div,
        btn("increment", 1),
        btn("decrement", -1),
        Node(:div, string(count), id="count"),
    )
  end
end

counter(1)

WebDisplay.Node{WebDisplay.Context}(WebDisplay.Context("context-10",IJuliaProvider(IJulia.CommManager.Comm{:webdisplay_comm}("8b76e8bf-974e-4cbb-8826-281ef1bb3dc9",true,#1,IJulia.CommManager.noop_callback)),Any[],Dict{Any,Any}(Pair{Any,Any}(:set_count,"function (ctx,msg) {ctx.dom.querySelector('#count').textContent = msg}"))),Persistent{WebDisplay.Node{WebDisplay.DOM}}[WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:div),Persistent{WebDisplay.Node{WebDisplay.DOM}}[WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:button),Persistent{String}[increment],Dict(:events=>Dict("click"=>"function (event,ctx) { WebDisplay.send(ctx, 'change', 1) }")),nothing,1), WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:button),Persistent{String}[decrement],Dict(:events=>Dict("click"=>"function (event,ctx) { WebDisplay.send(ctx, 'change', -1) }")),nothing,1), WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:div),Persistent{String}[1],Dict(:id=>"count"),nothing,1)],Dict{Any,Any}(),nothing,6)],Dict{Symbol,Any}(Pair{Symbol,Any}(:id,"context-10"),Pair{Symbol,Any}(:commands,Dict{Any,Any}(Pair{Any,Any}(:set_count,"function (ctx,msg) {ctx.dom.querySelector('#count').textContent = msg}"))),Pair{Symbol,Any}(:requires,Any[])),nothing,7)

In [7]:
Node(:div,
  map(counter, [1,2,3,4])
)

WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:div),Persistent{WebDisplay.Node{WebDisplay.Context}}[WebDisplay.Node{WebDisplay.Context}(WebDisplay.Context("context-5",IJuliaProvider(IJulia.CommManager.Comm{:webdisplay_comm}("8b76e8bf-974e-4cbb-8826-281ef1bb3dc9",true,#1,IJulia.CommManager.noop_callback)),Any[],Dict{Any,Any}(Pair{Any,Any}(:set_count,"function (ctx,msg) {ctx.dom.querySelector('#count').textContent = msg}"))),Persistent{WebDisplay.Node{WebDisplay.DOM}}[WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:div),Persistent{WebDisplay.Node{WebDisplay.DOM}}[WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:button),Persistent{String}[increment],Dict(:events=>Dict("click"=>"function (event,ctx) { WebDisplay.send(ctx, 'change', 1) }")),nothing,1), WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:button),Persistent{String}[decrement],Dict(:events=>Dict("click"=>"function (event,ctx) { WebDisplay.send(ctx, 'change', -1) }")),nothing,1), WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:div),Persistent{String}[1],Dict(:id=>"count"),nothing,1)],Dict{Any,Any}(),nothing,6)],Dict{Symbol,Any}(Pair{Symbol,Any}(:id,"context-5"),Pair{Symbol,Any}(:commands,Dict{Any,Any}(Pair{Any,Any}(:set_count,"function (ctx,msg) {ctx.dom.querySelector('#count').textContent = msg}"))),Pair{Symbol,Any}(:requires,Any[])),nothing,7), WebDisplay.Node{WebDisplay.Context}(WebDisplay.Context("context-6",IJuliaProvider(IJulia.CommManager.Comm{:webdisplay_comm}("8b76e8bf-974e-4cbb-8826-281ef1bb3dc9",true,#1,IJulia.CommManager.noop_callback)),Any[],Dict{Any,Any}(Pair{Any,Any}(:set_count,"function (ctx,msg) {ctx.dom.querySelector('#count').textContent = msg}"))),Persistent{WebDisplay.Node{WebDisplay.DOM}}[WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:div),Persistent{WebDisplay.Node{WebDisplay.DOM}}[WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:button),Persistent{String}[increment],Dict(:events=>Dict("click"=>"function (event,ctx) { WebDisplay.send(ctx, 'change', 1) }")),nothing,1), WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:button),Persistent{String}[decrement],Dict(:events=>Dict("click"=>"function (event,ctx) { WebDisplay.send(ctx, 'change', -1) }")),nothing,1), WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:div),Persistent{String}[2],Dict(:id=>"count"),nothing,1)],Dict{Any,Any}(),nothing,6)],Dict{Symbol,Any}(Pair{Symbol,Any}(:id,"context-6"),Pair{Symbol,Any}(:commands,Dict{Any,Any}(Pair{Any,Any}(:set_count,"function (ctx,msg) {ctx.dom.querySelector('#count').textContent = msg}"))),Pair{Symbol,Any}(:requires,Any[])),nothing,7), WebDisplay.Node{WebDisplay.Context}(WebDisplay.Context("context-7",IJuliaProvider(IJulia.CommManager.Comm{:webdisplay_comm}("8b76e8bf-974e-4cbb-8826-281ef1bb3dc9",true,#1,IJulia.CommManager.noop_callback)),Any[],Dict{Any,Any}(Pair{Any,Any}(:set_count,"function (ctx,msg) {ctx.dom.querySelector('#count').textContent = msg}"))),Persistent{WebDisplay.Node{WebDisplay.DOM}}[WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:div),Persistent{WebDisplay.Node{WebDisplay.DOM}}[WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:button),Persistent{String}[increment],Dict(:events=>Dict("click"=>"function (event,ctx) { WebDisplay.send(ctx, 'change', 1) }")),nothing,1), WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:button),Persistent{String}[decrement],Dict(:events=>Dict("click"=>"function (event,ctx) { WebDisplay.send(ctx, 'change', -1) }")),nothing,1), WebDisplay.Node{WebDisplay.DOM}(WebDisplay.DOM(:html,:div),Persistent{String}[3],Dict(:id=>"count"),nothing,1)],Dict{Any,Any}(),nothing,6)],Dict{Symbol,Any}(Pair{Symbol,Any}(:id,"context-7"),Pair{Symbol,Any}(:commands,Dict{Any,Any}(Pair{Any,Any}(:set_count,"function (ctx,msg) {ctx.dom.querySelector('#count').textContent = msg}"))),Pair{Symbol,Any}(:requires,Any[])),nothing,7), WebDisplay.Node{WebDisplay.Context}(WebDisplay.Context("context-8",IJuliaProvider(IJulia.CommManager.Comm{:webdisplay_comm}("8b76e8bf-974e-4cbb-8826-281ef1bb3